In [1]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 7.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [2]:
!pip install torchtext

In [ ]:
!pip install spacy

In [3]:
def generateBigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [4]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  preprocessing = generateBigrams)

LABEL = data.LabelField(dtype = torch.float)

In [7]:
import random

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(183))

In [8]:
MAX_VOCAB_SIZE = 30_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:39, 5.40MB/s]                           
100%|█████████▉| 398320/400000 [00:15<00:00, 25274.69it/s]

In [10]:
batch_size = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = batch_size, 
    device = device)

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
                
        embedded = self.embedding(text)
        embedded = embedded.permute(1, 0, 2)
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        return self.fc(pooled)

In [12]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [13]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 1.4716, -0.2535,  0.9360,  ...,  0.8706, -0.8580, -0.7130],
        [-1.5451,  1.4988,  0.9197,  ..., -1.0257,  2.0295, -0.6137],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-1.6321, -0.4962,  0.2880,  ...,  0.1996, -0.7337, -1.5082],
        [ 0.6045,  0.5900, -0.1939,  ..., -0.4255,  0.4184,  0.4414],
        [ 0.0924,  0.4106, -0.9514,  ...,  0.1037,  1.0813, -0.8338]])

In [14]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Train the Model

In [15]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [16]:
def binaryAccuracy(preds, y):
    # Returns accuracy per batch and always round predictions to closest int
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [17]:
def train_fast_text(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binaryAccuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
def evaluate_fast_text(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binaryAccuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [19]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Training model

In [22]:
epochs = 7

best_valid_loss = float('inf')

for epoch in range(epochs):

    start_time = time.time() #start time
    
    train_loss, train_acc = train_fast_text(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_fast_text(model, valid_iterator, criterion)
    
    end_time = time.time() #end time

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'ft_model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 10s
	Train Loss: 0.577 | Train Acc: 79.64%
	 Val. Loss: 0.385 |  Val. Acc: 82.39%
Epoch: 02 | Epoch Time: 0m 10s
	Train Loss: 0.454 | Train Acc: 86.10%
	 Val. Loss: 0.333 |  Val. Acc: 86.18%
Epoch: 03 | Epoch Time: 0m 10s
	Train Loss: 0.370 | Train Acc: 88.83%
	 Val. Loss: 0.335 |  Val. Acc: 87.59%
Epoch: 04 | Epoch Time: 0m 10s
	Train Loss: 0.311 | Train Acc: 90.53%
	 Val. Loss: 0.349 |  Val. Acc: 88.52%
Epoch: 05 | Epoch Time: 0m 9s
	Train Loss: 0.271 | Train Acc: 91.81%
	 Val. Loss: 0.366 |  Val. Acc: 89.04%
Epoch: 06 | Epoch Time: 0m 9s
	Train Loss: 0.240 | Train Acc: 92.60%
	 Val. Loss: 0.384 |  Val. Acc: 89.27%
Epoch: 07 | Epoch Time: 0m 9s
	Train Loss: 0.216 | Train Acc: 93.47%
	 Val. Loss: 0.396 |  Val. Acc: 89.79%


In [24]:
model.load_state_dict(torch.load('ft_model.pt'))

test_loss, test_acc = evaluate_fast_text(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.356 | Test Acc: 85.59%


## User Input



In [25]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generateBigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [26]:
predict_sentiment(model, "This film is not terrible")

0.999991774559021

In [27]:
predict_sentiment(model, "This film is great")

1.308610151534608e-17